In [1]:
import os
import wave
import struct
import threading
import torch
import librosa
import tkinter as tk

from pydub import AudioSegment
from pvrecorder import PvRecorder
from pydub.silence import split_on_silence
from transformers import Wav2Vec2Tokenizer, Wav2Vec2ForCTC


d:\MiniConda\envs\tf\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
file_path = r'C:/Users/HI/Desktop/Wave2Text/new_file'
if not os.path.exists(file_path):
    os.makedirs(file_path)

In [3]:
tokenizer = Wav2Vec2Tokenizer.from_pretrained("nguyenvulebinh/wav2vec2-base-vietnamese-250h")
model = Wav2Vec2ForCTC.from_pretrained("nguyenvulebinh/wav2vec2-base-vietnamese-250h")

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'Wav2Vec2CTCTokenizer'. 
The class this function is called from is 'Wav2Vec2Tokenizer'.
d:\MiniConda\envs\tf\lib\site-packages\transformers\models\wav2vec2\tokenization_wav2vec2.py:792: FutureWarning: The class `Wav2Vec2Tokenizer` is deprecated and will be removed in version 5 of Transformers. Please use `Wav2Vec2Processor` or `Wav2Vec2CTCTokenizer` instead.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
d:\MiniConda\envs\tf\lib\site-packages\transformers\configuration_utils.py:380: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer

In [4]:
class VoiceRecorder:
    
    def __init__(self):
        self.root = tk.Tk()
        self.root.resizable(False, False)
        self.button = tk.Button(text = "🎤", font = ("Arial", 120, "bold"), command = self.click_handler)
        self.button.pack()
        self.output = tk.Text(self.root, height = 5, width = 41,)
        self.output.pack()
        self.recording = False
        self.root.mainloop()
    
    def click_handler(self):
        if self.recording:
            self.recording = False
            self.button.config(fg = "black")
        else:
            self.recording = True
            self.button.config(fg = "red")
            threading.Thread(target = self.record).start()
            
    
    def delete_files_in_directory(self, directory_path):
        try:
            files = os.listdir(directory_path)
            for file in files:
                file_path = os.path.join(directory_path, file)
                if os.path.isfile(file_path):
                    os.remove(file_path)
            print("All files deleted successfully.")
        except OSError:
            print("Error occurred while deleting files.")

    def speech2text(self, directory_path):
        data, _ = librosa.load(directory_path, sr=16000)
        input_values = tokenizer(data, return_tensors = "pt").input_values
        logits = model(input_values).logits
        prediction = torch.argmax(logits, dim = -1)
        transcription = tokenizer.batch_decode(prediction)[0]

        return transcription
    def record(self):
        recorder = PvRecorder(device_index=0, frame_length=512)
        audio = []
        
        recorder.start()
        
        while self.recording:
            frame = recorder.read()
            audio.extend(frame)
        
        recorder.stop()
        with wave.open(file_path + '\sound.wav', 'w') as f:
            f.setparams((1, 2, 16000, 512, "NONE", "NONE"))
            f.writeframes(struct.pack("h" * len(audio), *audio))
        
        transcription = self.speech2text(file_path+"\sound.wav")
        
        self.output.insert("1.0", transcription + '\n')
        self.delete_files_in_directory(file_path)
        recorder.delete()

VoiceRecorder()

All files deleted successfully.
All files deleted successfully.
All files deleted successfully.
All files deleted successfully.
All files deleted successfully.
All files deleted successfully.
All files deleted successfully.
All files deleted successfully.
All files deleted successfully.
